# 1. Import Dependencies

In [1]:
#!pip install stable-baselines3[extra]

In [2]:
import stable_baselines3


In [51]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

# 3. Building an Environment

In [52]:
class ShowerEnv(Env):
    def __init__(self):
        
        # number of pentomino pieces
        no_pieces = 9 # + 1 + 1 #pieces + uncertainty + lang team
        
        self.uncertainty_action = 10
        self.lang_action = 11
        
        # ACTIONS
        '''for now: we only decide for one coordinate --> no of actions = number of coordinates)
        (for later: we only decide for one coordinate out of all absolute coordinates and moving coordinate, 
        boolean value expressing uncertainty)'''
        self.action_space = Discrete(no_pieces)
        
        # Temperature array
        self.observation_space = Box(low=0, high=100, shape=(no_pieces,))
        
        # POSSIBLE STATES
        '''Set start state (in our case start state is the only state and here we need to get our training data in or 
        construct a random formula that will randomly generate possible scenarios each time the function is called.
        for now: create random probabilities for 9 coordinates that add up to one
        uplevel: do the same and make sure that all follow the true distribution which is p=1 for the gold_coordinate and p=0 for all others
        uplevel: do the same + add possiblity for moving vector'''
        bleh = [random.uniform(0, 0.628) for _ in range(8)]
        bleh.append(random.uniform(2, 5))

        self.state = np.exp(bleh)/np.sum(np.exp(bleh))
        
#         bleh = [random.uniform(0, 0.628) for _ in range(8)]
#         bleh.append(random.uniform(2, 5))
#         random.seed(1)
#         random.shuffle(bleh)
#         bleh = np.exp(bleh)/np.sum(np.exp(bleh))
#         bleh_dict1 = {idx:i for idx, i in enumerate(bleh)}
        
#         self.language_and_vision = {'confidence_instruction': random.uniform(0, 1), 'coordinates': bleh_dict1}
        
#         bleh = [random.uniform(0, 0.628) for _ in range(8)]
#         bleh.append(random.uniform(2, 5))
#         random.seed(1)
#         random.shuffle(bleh)
#         bleh = np.exp(bleh)/np.sum(np.exp(bleh))
#         bleh_dict2 = {idx:i for idx, i in enumerate(bleh)}
        
#         self.gesture = {'confidence_instruction': random.uniform(0, 1), 'coordinates': bleh_dict2}

        
#         self.language = {'confidence_instruction': random.uniform(0, 1)}
        
        

        
    def step(self, action):
        '''Our actions do not affect our state, because we only have one state, the start state. Also we don't need 
        the shower_length which represents the lenght of the sequence, but at the end we use it in the evalulation-
        function soo .. dunno.
        '''
        
        # Calculate reward 
        '''put hard-coded rules in!!!'''        
#         gold_label = -1
#         instr_thresh = 0.90
#         coord_thresh = 0.80

#         if self.language['confidence_instruction'] > instr_thresh:
#             gold_label = self.lang_action
#         else:
#             if self.language_and_vision['confidence_instruction'] > instr_thresh > self.gesture['confidence_instruction']:
#                 coordinate = max(self.language_and_vision['coordinates'], key=self.language_and_vision['coordinates'].get)
#                 probability = self.language_and_vision['coordinates'][coordinate]
#                 if probability > coord_thresh:
#                     gold_label = coordinate
#                 else:
#                     gold_label = self.uncertainty_action
#             elif self.language_and_vision['confidence_instruction'] < instr_thresh < self.gesture['confidence_instruction']:
#                 print(self.gesture['coordinates'])
#                 coordinate = max(self.gesture['coordinates'], key=self.gesture['coordinates'].get)
#                 probability = self.gesture['coordinates'][coordinate]
#                 if probability > coord_thresh:
#                     gold_label = coordinate
#                 else:
#                     gold_label = self.uncertainty_action
#             elif self.language_and_vision['confidence_instruction'] > instr_thresh and self.gesture['confidence_instruction'] > instr_thresh:
#                 mean = dict()
#                 for key in self.language_and_vision['coordinates']:
#                     mean[key] = (self.language_and_vision['coordinates'][key]+self.gesture['coordinates'][key])/2
#                 max_mean = max(mean, key=mean.get)
#                 if mean[max_mean] > coord_thresh:
#                     gold_label = max_mean
#                 else:
#                     gold_label = self.uncertainty_action
#             elif self.language_and_vision['confidence_instruction'] < instr_thresh and self.gesture['confidence_instruction'] < instr_thresh:
#                     gold_label = self.uncertainty_action
            
#         if action == gold_label:
#             reward = 1
#         else:
#             reward = -1

        if action == np.argmax(self.state):
            reward = 1
        else:
            reward = -1
        
        # Check if sequence is done
        '''we have a sequence of one state: the output that the other groups give us. based on that 
        we make a decision and a new round starts. so we don't need to count down the seconds of a 60min long
        shower to know when a round is over.'''
        done = True
        
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        '''Reset start state (output of previous groups) when new round starts (use "random-formula" that we need to 
        create above)'''
        bleh = [random.uniform(0, 0.628) for _ in range(8)]
        bleh.append(random.uniform(2, 5))

        self.state = np.exp(bleh)/np.sum(np.exp(bleh))

        return self.state

In [53]:
env=ShowerEnv()

In [54]:
#CLARA
env.observation_space 
env.observation_space.sample()
env.action_space.sample()

7

In [55]:
env.reset()

array([0.09375583, 0.05426112, 0.05408611, 0.07478403, 0.09600335,
       0.06762608, 0.06098463, 0.0693861 , 0.42911276])

# 4. Test Environment

In [56]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-1
Episode:2 Score:-1
Episode:3 Score:-1
Episode:4 Score:-1
Episode:5 Score:-1


In [57]:
env.close()

# 5. Train Model

In [58]:
log_path = os.path.join('Training', 'Logs')

In [59]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

# Random Agent, before training   
'''stolen from: https://colab.research.google.com/github/Stable-Baselines-Team/rl-colab-notebooks/blob/master/stable_baselines_getting_started.ipynb#scrollTo=xDHLMA6NFk95
since the evaluate_policy function from this tutorial errored'''
evaluate_policy(model, env, n_eval_episodes=100)            

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


(1.0, 0.0)

In [60]:
model.learn(total_timesteps=100000)

Logging to Training/Logs/PPO_7
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -0.82    |
| time/              |          |
|    fps             | 2803     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1          |
|    ep_rew_mean          | -0.34      |
| time/                   |            |
|    fps                  | 1778       |
|    iterations           | 2          |
|    time_elapsed         | 2          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.19937497 |
|    clip_fraction        | 0.745      |
|    clip_range           | 0.2        |
|    entropy_loss         | -2.01      |
|    explained_variance   | -0.00472   |
|    learning_

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | 1         |
| time/                   |           |
|    fps                  | 1388      |
|    iterations           | 11        |
|    time_elapsed         | 16        |
|    total_timesteps      | 22528     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.00403  |
|    explained_variance   | nan       |
|    learning_rate        | 0.0003    |
|    loss                 | 1.68e-05  |
|    n_updates            | 100       |
|    policy_gradient_loss | -1.33e-08 |
|    value_loss           | 4.16e-05  |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | 1         |


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | 1         |
| time/                   |           |
|    fps                  | 1363      |
|    iterations           | 21        |
|    time_elapsed         | 31        |
|    total_timesteps      | 43008     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.00112  |
|    explained_variance   | nan       |
|    learning_rate        | 0.0003    |
|    loss                 | 1.35e-06  |
|    n_updates            | 200       |
|    policy_gradient_loss | -2.22e-08 |
|    value_loss           | 3.76e-06  |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | 1         |


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | 1         |
| time/                   |           |
|    fps                  | 1356      |
|    iterations           | 31        |
|    time_elapsed         | 46        |
|    total_timesteps      | 63488     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.000549 |
|    explained_variance   | nan       |
|    learning_rate        | 0.0003    |
|    loss                 | 8.77e-07  |
|    n_updates            | 300       |
|    policy_gradient_loss | -2.86e-08 |
|    value_loss           | 1.77e-06  |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | 1         |


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | 1         |
| time/                   |           |
|    fps                  | 1351      |
|    iterations           | 41        |
|    time_elapsed         | 62        |
|    total_timesteps      | 83968     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.000254 |
|    explained_variance   | nan       |
|    learning_rate        | 0.0003    |
|    loss                 | 5.44e-07  |
|    n_updates            | 400       |
|    policy_gradient_loss | 1.42e-08  |
|    value_loss           | 1.2e-06   |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | 1         |


# 6. Save Model

In [61]:
#ORIGINAL
#model.save('PPO')

In [62]:
#ORIGINAL
#evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [63]:
# FROM YT - CLARA
shower_path = os.path.join('Training', 'Saved Models', 'Shower_Model_PPO')


In [64]:
model.save(shower_path)

In [65]:
del model

In [66]:
model = PPO.load(shower_path,env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [67]:
#evaluate_policy(model,env,n_eval_episodes=10,render=True)

In [68]:
# Random Agent, after training                            
'''stolen from: https://colab.research.google.com/github/Stable-Baselines-Team/rl-colab-notebooks/blob/master/stable_baselines_getting_started.ipynb#scrollTo=xDHLMA6NFk95
since the evaluate_policy function from this tutorial errored'''
evaluate_policy(model, env, n_eval_episodes=100)           

(1.0, 0.0)

In [69]:
'''MODEL ACTUALLY LEARNS!

(mean shower length, variance in shower length)
(however i dont know why we re interested in shower length since the goal of our agent is to regulate the temperature)
total_timesteps = 20 000: (-57.76, 2.486443242867209)
total_timesteps = 100 000:(59.38, 0.9249864863877744)'''

'MODEL ACTUALLY LEARNS!\n\n(mean shower length, variance in shower length)\n(however i dont know why we re interested in shower length since the goal of our agent is to regulate the temperature)\ntotal_timesteps = 20 000: (-57.76, 2.486443242867209)\ntotal_timesteps = 100 000:(59.38, 0.9249864863877744)'

In [70]:
bleh = np.random.randn(1, 9)
sft = np.exp(bleh)/np.sum(np.exp(bleh))
print(np.argmax(sft))
print(np.max(sft))

8
0.6718757921255092


In [71]:
bleh = [random.uniform(0, 0.628) for _ in range(8)]
bleh.append(random.uniform(2, 5))

sft = np.exp(bleh)/np.sum(np.exp(bleh))
print(bleh)
random.shuffle(bleh)
print(bleh)
# print(sft)
# print(np.argmax(sft))
# print(np.max(sft))

[0.4181578438981117, 0.16752103886134737, 0.05816203372130036, 0.45717501774503905, 0.029367739107433857, 0.4089619802177785, 0.0805606185802519, 0.1644834166624538, 3.7497764171024635]
[0.1644834166624538, 0.4089619802177785, 3.7497764171024635, 0.45717501774503905, 0.16752103886134737, 0.029367739107433857, 0.0805606185802519, 0.4181578438981117, 0.05816203372130036]


In [72]:
print(sft)

[0.02885377 0.02245703 0.02013067 0.03000181 0.01955929 0.02858965
 0.02058666 0.02238892 0.8074322 ]
